In [ ]:
from collections import defaultdict, OrderedDict
import datetime
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from geopy.distance import vincenty
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import euclidean_distances
import time

%matplotlib inline

# Turn off chained assignment warning
pd.options.mode.chained_assignment = None

In [ ]:
INPUT_PATH = 'inputs/'
OUTPUT_PATH = 'outputs_1/'


class Data(object):
    pass

results = Data()

# Load data

In [ ]:
flood_data = pd.read_csv(INPUT_PATH + '311_2015_flooding.csv', parse_dates=[1])

In [ ]:
flood_data.head()

In [ ]:
flood_data.shape

In [ ]:
flood_data.dtypes

In [ ]:
results.n_311_flooding_reports = flood_data.shape[0]

# Analysis

In [ ]:
# number of reports of flooding per day required to qualify that day as part of a storm period
storm_period_threshold = 50


In [ ]:
# How many days of the year have flooding data?
flood_data.created_date.dt.dayofyear.nunique()

In [ ]:
def plot_location():
    
    data = flood_data#[flood_data.created_date.dt.weekofyear == day_of_year]
    
    plt.plot(data.latitude, data.longitude, '.', alpha=0.1)

plot_location()

In [ ]:
def plot_flood_reports(self):
    # NB A couple days might be missing from this plot because they had zero reports of flooding
    plt.figure(figsize=(8, 2))
    plt.plot(self.flood_data.groupby(self.flood_data.created_date.dt.dayofyear).size(), '.-')
    plt.axhline(self.storm_period_threshold, color='black', linestyle='dashed')
    plt.xlabel('Day of year')
    plt.xlim(0, 365)
    plt.ylabel('Number of flooding reports')
    plt.savefig('flood_reports.png', dpi=200, bbox_inches='tight')

In [ ]:
plot_flood_reports()

In [ ]:
def get_storm_periods():
    """
    In the first step, consecutive days with more than {storm_period_threshold} reports of flooding were classed as
    storm periods. For {data_year}, we identified {number_of_storm_periods} storms.
    """
    
    is_storm_periods = flood_data.groupby(flood_data.created_date.dt.dayofyear).size() > storm_period_threshold
    
    storm_periods = defaultdict(list)
    i = 0
    is_yesterday = False
    
    for today, is_today in zip(is_storm_periods.index, is_storm_periods):
        
        # if yesterday was False, a new run is beginning
        if not is_today and is_yesterday:
            i += 1
        
        if is_today:
            storm_periods[i].append(today)
            
        is_yesterday = is_today
    
    results.number_of_storm_periods = len(storm_periods)
    return storm_periods

In [ ]:
@add_step
def eps_to_miles(self):
    
    latitude_max = self.flood_data.latitude.max()
    latitude_min = self.flood_data.latitude.min()
    longitude_max = self.flood_data.longitude.max()
    longitude_min = self.flood_data.longitude.min()
    
    #print(list(zip((latitude_max, longitude_min), 
    #         (latitude_min, longitude_min), 
    #         (latitude_max, longitude_min), 
    #         (latitude_max, longitude_max))))
    
    #plt.plot(*list(zip(
    #         (latitude_max, longitude_min), 
    #         (latitude_min, longitude_min), 
    #         (latitude_max, longitude_min), 
    #         (latitude_max, longitude_max))), '.')
    #plt.xlabel('latitude')
    #plt.ylabel('longitude')
    
    distance_lat_miles = vincenty((latitude_max, longitude_min), (latitude_min, longitude_min)).miles
    distance_lat_units = latitude_max - latitude_min
    
    distance_long_miles = vincenty((latitude_max, longitude_min), (latitude_max, longitude_max)).miles
    distance_long_units = longitude_max - longitude_min
    
    self.miles_per_deg_lat = distance_lat_miles/distance_lat_units
    self.miles_per_deg_long = distance_long_miles/distance_long_units

In [ ]:
@add_step
def cluster_storms_tune_eps(self):

    self.n_floods_by_eps = []
    self.n_outliers_by_eps = []
    self.eps_range = np.arange(0.00001, 0.050, 0.001)
    
    for eps in self.eps_range:
    
        self.cluster_storms(eps)
        self.n_outliers_by_eps.append(self.n_outliers)
        self.n_floods_by_eps.append(self.n_floods)

    self.eps_range_miles = self.miles_per_deg_lat * self.eps_range

@add_step
def plot_n_floods_n_outliers_by_eps(self):
    
    plt.plot(self.eps_range_miles, self.n_floods_by_eps)
    plt.xlabel('Neighborhood radius in miles*')
    plt.ylabel('Number of flood clusters')
    plt.axvline(0.25, linestyle='dashed', color='black')
    plt.twinx()
    # spoof a line to get the label added to the legend
    plt.plot(np.nan, label='Flood clusters')
    plt.plot(self.eps_range_miles, self.n_outliers_by_eps, color='green', label='Outliers')
    plt.ylabel('Number of outlier points')
    plt.legend()
    plt.savefig('n_floods_n_outliers_by_esp.png', dpi=200, bbox_inches='tight')

In [ ]:
def cluster_attributes(cluster):
    
    latitude_max = cluster.latitude.max()
    latitude_min = cluster.latitude.min()
    longitude_max = cluster.longitude.max()
    longitude_min = cluster.longitude.min()
    
    space_diameter = vincenty((latitude_max, longitude_min), (latitude_min, longitude_max)).miles    
   
    time_diameter = cluster.created_date.max() - cluster.created_date.min()
    
    time_start = cluster.created_date.min()
    time_end = cluster.created_date.max()
    
    space_center_latitude = cluster.latitude.mean()
    space_center_longitude = cluster.longitude.mean()    

    return [time_start, time_end, space_center_latitude, space_center_longitude,
            space_diameter, time_diameter, cluster.shape[0]]

In [ ]:
def cluster_storms(eps):
    
    self.floods = []
    self.eps = eps
    self.minimum_flood_reports = 3
    self.n_outliers = 0
    
    for storm_number, storm_days in self.storm_periods.items():
        
        storm_floods = self.flood_data[self.flood_data.created_date.dt.dayofyear.isin(set(storm_days))]
        storm_floods_matrix = storm_floods[['latitude', 'longitude']].as_matrix()
        
        #print(storm_floods_matrix)
        dbscan = DBSCAN(eps=self.eps, min_samples=self.minimum_flood_reports)
        
        storm_floods['cluster'] = dbscan.fit_predict(storm_floods_matrix)
        
        is_outlier = (storm_floods.cluster == -1)
        self.n_outliers += is_outlier.sum()
        storm_floods = storm_floods[~is_outlier]
        
        floods = list(storm_floods.groupby('cluster').apply(cluster_attributes).values)
        floods = [flood + [storm_number] for flood in floods]
        #print(floods)
        self.floods.extend(floods)
    
    
    self.floods = pd.DataFrame(self.floods,
             columns=['time_start', 'time_end', 'latitude_center', 'longitude_center',
                     'diameter', 'duration', 'number_of_reports', 'storm_number'])
    self.n_floods = len(self.floods)
    
    return floods, n_outliers

In [ ]:
e.get_storm_periods()

In [ ]:
e.cluster_storms_tune_eps()

In [ ]:
e.eps_to_miles()

In [ ]:
e.plot_n_floods_n_outliers_by_eps()

In [ ]:
e.cluster_storms(0.25/e.miles_per_deg_long)
e.n_floods

In [ ]:
plt.plot(e.flood_data['longitude'], e.flood_data['latitude'], '.', alpha=0.2)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.savefig('flood_reports_by_location.png', dpi=200, bbox_inches='tight')

In [ ]:
plt.plot(e.floods['longitude_center'], e.floods['latitude_center'], 'x', alpha=0.8)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.savefig('flood_clusters_by_location.png', dpi=200, bbox_inches='tight')

In [ ]:
#e.eps_to_miles()
#e.get_storm_periods()
e.cluster_storms(0.004)